In [ ]:
import random

class Strategy():
    '''A class containing the genome for a strategy, as well as its fitness.'''
    def __init__(self, length, actions):
        #every number will correspond to an action
        self.length = length
        self.actions = actions
        self.genome = [random.randint(0, self.actions-1) for i in range(self.length)]
        self.fitness = None
        
    def __repr__(self):
        return str(self.genome)

class ChildStrategy(Strategy):
    '''A child strategy will create a child of the 2 parent strategies, with some random genes'''
    def __init__(self, parent1, parent2):
        if not (isinstance(parent1, Strategy) and isinstance(parent2, Strategy)):
            raise TypeError('parents\' type must be Strategy()')
        if parent1.length != parent2.length:
            raise TypeError('parents must have the same number of situations')
        if parent1.actions != parent2.actions:
            raise TypeError('parents must have the same number of actions')
        self.actions = parent1.actions
        self.length = parent1.length
        self.fitness = None
        self.genome = []
        for i in range(self.length):
            prob = random.random()
            # you could change these parameters to change the proportions of genes contributed by each parent and also the proportion of random genes
            if prob < 0.45:
                self.genome.append(parent1.genome[i])
            elif prob < 0.90:
                self.genome.append(parent2.genome[i])
            else:
                self.genome.append(random.randint(0,self.actions))

In [ ]:
import pandas as pd
import os

class CurrentState():
    '''A class containing all the relevant information about the current state of the simulation'''
    def __init__(self, strategy):
        # the following is a sloppy way of finding a a ticker whose file actually exists
        with open('backtesting_data/tickers.txt') as tickers_files:
            tickers = [i.strip() for i in tickers_files]
            found = False
            while not found:
                index = random.randint(0, len(tickers) - 1)
                self.ticker = tickers[index]
                if os.path.isfile('backtesting_data/data/' + self.ticker + '.csv'):
                    found = True
        # here all of the main attributes are initialised
        self.strategy = strategy
        self.df = pd.read_csv('backtesting_data/data/' + self.ticker + '.csv')
        self.day = random.randint(1, len(self.df) - 1001)
        self.date = self.df.at[self.day, 'Date']
        self.liquidity = 100.0
        self.shares = 0.0
        self.high_y = self.df.at[self.day-1, 'High']
        self.high_t = self.df.at[self.day, 'High']
        self.low_y = self.df.at[self.day-1, 'Low']
        self.low_t = self.df.at[self.day, 'Low']
        self.close_t = self.df.at[self.day, 'Close']
        self.open_t = self.df.at[self.day, 'Open']
        self.investment = self.shares * self.open_t
        self.total_assets = self.liquidity + self.investment
    
    def next_day(self):
        #all of the main attributes are updated to the next day
        self.day += 1
        self.date = self.df.at[self.day, 'Date']
        self.high_y = self.df.at[self.day-1, 'High']
        self.high_t = self.df.at[self.day, 'High']
        self.low_y = self.df.at[self.day-1, 'Low']
        self.low_t = self.df.at[self.day, 'Low']
        self.close_t = self.df.at[self.day, 'Close']
        self.open_t = self.df.at[self.day, 'Open']
        self.investment = self.shares * self.open_t
        self.total_assets = self.liquidity + self.investment
    
    # the folllowing functions implement all of the actions the investor can take
    def _buy_all(self):
        if self.close_t == 0:
            return 'buy nothing'
        newshares = self.liquidity / self.close_t
        self.shares += newshares
        self.liquidity = 0.0
        return f'bought {newshares} shares (all) at {self.close_t}'
    
    def _buy_half(self):
        if self.close_t == 0:
            return 'buy nothing'
        newshares = (self.liquidity / 2) / self.close_t
        self.shares += newshares
        self.liquidity /= 2
        return f'bought {newshares} shares (half) at {self.close_t}'
    
    def _hold(self):
        return 'nothing'
    
    def _sell_half(self):
        soldshares = self.shares/2
        self.liquidity += soldshares * self.close_t
        self.shares /= 2
        return f'sold {soldshares} shares (half) at {self.close_t}'
    
    def _sell_all(self):
        soldshares = self.shares
        self.liquidity += self.shares * self.close_t
        self.shares = 0.0
        return f'sold {soldshares} shares (all) at {self.close_t}'
    
   
    def _recognize_situation(self):
        # this function translates boolean values into a decimal number from 0 to 15
        # which corresponds to an index in the genome list
        situation = -1
        if self.liquidity > self.investment:
            situation += 8
        if self.high_y > self.high_t:
            situation += 4
        if self.low_y > self.low_t:
            situation += 2
        if self.close_t > self.open_t:
            situation += 1
        return situation
    
    
    def act(self):
        # this function translates the index to an action taken by the investor
        situation = self._recognize_situation()
        action = self.strategy.genome[situation]
        if action == 0:
            return self._buy_all()
        elif action == 1:
            return self._buy_half()
        elif action == 2:
            return self._hold()
        elif action == 3:
            return self._sell_half()
        elif action == 4:
            return self._sell_all()
            

def test_strategy(strategy):
    # this function tests the strategy for 1000 days
    state = CurrentState(strategy)
    for i in range(1000):
        state.act()
        state.next_day()
    return(state.total_assets)

strat = Strategy(16,5)
print(test_strategy(strat))

In [ ]:
import numpy as np
import pickle

def save_object(obj):
    try:
        with open("data.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

class Generation():
    def __init__(self, population, total_fitness, fitness_list):
        self.population = population
        self.total_fitness = total_fitness
        self.fitness_list = fitness_list
    
    def __repr__(self):
        return f'pop: {self.population}\nfitness: {self.total_fitness}\n'

generations = []

population = [Strategy(16,5) for i in range(100)]

for generation in range(300):
    for strategy in population:
        fitness_list = [test_strategy(strategy) for i in range(100)]
        strategy.fitness = sum(fitness_list) / 100
    
    generation_fitness = [strategy.fitness for strategy in population]
    total_fitness = sum(generation_fitness)
    strategy_probabilities = [strategy.fitness/total_fitness for strategy in population]
    
    generations.append(Generation(population, total_fitness, generation_fitness))
    
    nextgen = []
    for i in range(len(population)):
        parent1 = np.random.choice(population, p = strategy_probabilities)
        parent2 = np.random.choice(population, p = strategy_probabilities)
        nextgen.append(ChildStrategy(parent1,parent2))
    population = nextgen[:]
    print(f'generation {generation} done\naverage fitness: {total_fitness/100}')

save_object(generations)